In [ ]:
import ultralytics
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')
import torch
import cv2
import numpy as np
from PIL import Image
from scipy.sparse.linalg import svds
from UVSM.utils.image import show_cam_on_image
import os

def get_2d_projection(activation):
    """
    Compute 2D projection of activations using SVD
    """
    # Get activation shape
    activation_shape = activation.shape
    print(f"Activation shape before reshaping: {activation_shape}")
    
    # Remove batch dimension and reshape channels as features
    # From (1, C, H, W) to (H*W, C)
    reshaped_activation = activation[0].transpose(1, 2, 0).reshape(-1, activation.shape[1])
    print(f"Reshaped activation shape: {reshaped_activation.shape}")
    
    # Center the data
    reshaped_activation = reshaped_activation - reshaped_activation.mean(axis=0)
    
    # Compute valid k value
    min_dim = min(reshaped_activation.shape)
    k = max(1, min(min_dim - 1, 3))
    print(f"Using k={k} for SVD (min_dim={min_dim})")
    
    if min_dim <= 1:
        raise ValueError(f"Invalid activation shape for SVD: {reshaped_activation.shape}")
    
    # Perform SVD
    U, S, VT = svds(reshaped_activation.astype(np.float64), k=k)
    
    # Project onto principal component
    projection = reshaped_activation @ VT[-1, :]
    
    # Reshape back to original spatial dimensions (H, W)
    return projection.reshape(activation_shape[2], activation_shape[3])

class ModifiedEigenCAM:
    def __init__(self, model, target_layers, task='od'):
        self.model = model
        self.target_layers = target_layers
        self.task = task
    
    def __call__(self, input_image):
        # Convert input to tensor if needed
        if isinstance(input_image, np.ndarray):
            input_tensor = torch.from_numpy(input_image).permute(2, 0, 1).unsqueeze(0).float()
            input_tensor = input_tensor / 255.0
        
        # Get activations
        activations = []
        def hook_fn(module, input, output):
            activations.append(output.detach().cpu().numpy())
        
        handles = []
        for layer in self.target_layers:
            handles.append(layer.register_forward_hook(hook_fn))
        
        # Forward pass
        with torch.no_grad():
            _ = self.model(input_tensor)
        
        # Remove hooks
        for handle in handles:
            handle.remove()
        
        # Process each activation
        result = []
        for activation in activations:
            try:
                cam = get_2d_projection(activation)
                result.append(cam)
            except Exception as e:
                print(f"Error processing activation: {e}")
                print(f"Activation shape: {activation.shape}")
                continue
        
        return np.float32(result)

def generate_heatmaps(model_path, image_path, output_dir=None, num_layers=20, image_size=(640, 640)):
    """
    Generate heatmaps for YOLO model layers using modified EigenCAM
    """
    # Load model
    model = YOLO(model_path)
    model.cpu()
    
    # Prepare input image
    img = cv2.imread(image_path)
    img = cv2.resize(img, image_size)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_float = np.float32(rgb_img) / 255
    
    # Setup output directory
    if output_dir is None:
        name = os.path.splitext(os.path.basename(image_path))[0]
        output_folder = r"C:\Users\MaxwellLee\PycharmProjects\UVSM\runs\heatmaps"
        output_dir = os.path.join(output_folder, name)
    os.makedirs(output_dir, exist_ok=True)
    
    # Get model layers
    all_layers = list(model.model.model)
    layers_to_process = all_layers[-num_layers:] if len(all_layers) > num_layers else all_layers
    
    for i, layer in enumerate(layers_to_process):
        print(f"\nProcessing Layer {i+1}/{len(layers_to_process)}: {layer.__class__.__name__}")
        
        try:
            # Generate heatmap
            cam = ModifiedEigenCAM(model.model, [layer], task='od')
            grayscale_cams = cam(rgb_img)
            
            if len(grayscale_cams) == 0:
                print(f"No valid heatmap generated for layer {i+1}")
                continue
            
            grayscale_cam = grayscale_cams[0]
            
            # Normalize heatmap
            grayscale_cam = np.maximum(grayscale_cam, 0)
            grayscale_cam = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min() + 1e-8)
            
            # Resize heatmap to match image size
            grayscale_cam = cv2.resize(grayscale_cam, image_size)
            
            # Apply heatmap to image
            cam_image = show_cam_on_image(img_float, grayscale_cam, use_rgb=True)
            
            # Save the comparison image
            output_path = os.path.join(output_dir, f"layer_{i+1}_{layer.__class__.__name__}_heatmap.jpg")
            comparison = np.hstack((rgb_img, cam_image))
            Image.fromarray(comparison).save(output_path)
            print(f"Saved heatmap to {output_path}")
            
        except Exception as e:
            print(f"Warning: Could not generate heatmap for layer {i+1}: {str(e)}")
            continue
            
    print("\nHeatmap generation complete!")
    
if __name__ == "__main__":
    model_path = "Path_To_Your_Model"  # Update with your model path
    image_path = "Path_To_Your_Image"  # Update with your image path
    
    generate_heatmaps(
        model_path=model_path,
        image_path=image_path,
        num_layers=20,
        image_size=(640, 640)  # Updated to match your input size
    )

In [ ]:
# ################## SHOW ALL LAYERS OF SAVED HEATMAP (ASCENDING ORDER) ##################

import os
import cv2
import matplotlib.pyplot as plt

def extract_numeric_part(filename):
    # Extract numeric part from the filename (assuming the numeric part is at the end of the filename before extension)
    return int(''.join(filter(str.isdigit, filename)))  # Extract digits and convert to integer

image_path = r"Path_To_Your_Image"

# List all files in the 'heatmaps' directory
image_files = os.listdir(image_path)

# Sort files numerically using the custom sorting function
image_files.sort(key=extract_numeric_part)

# Loop through each image file in the directory
for image_name in image_files:
    image_file_path = os.path.join(image_path, image_name)  # Create full path to the image file
    if os.path.isfile(image_file_path):  # Check if it's a file
        im = cv2.imread(image_file_path)  # Read the image
        print(f"{image_file_path}")

        if im is not None:
            # Convert BGR to RGB (OpenCV loads images in BGR by default)
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            
            # Set the figure size to enlarge the image (width, height in inches)
            plt.figure(figsize=(12.8, 6.4))  # You can adjust these values to your preference
            
            # Display image using matplotlib
            plt.imshow(im_rgb)
            plt.axis('off')  # Hide axes
            plt.show()  # Show the image in the notebook
        else:
            print(f"Failed to load image: {image_file_path}")